<a href="https://www.kaggle.com/code/gowthammani/cyclone-intensity?scriptVersionId=140194620" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat_3d_ds - Sheet.csv
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/35(2).jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/55(2).jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/36(3).jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/45.jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/56.jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/58.jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/44(2).jpg
/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED/46(3).jp

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNetV2,EfficientNetB0,EfficientNetB4,Xception
import glob
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
train = pd.read_csv("../input/insat3d-infrared-raw-cyclone-images-20132021/insat_3d_ds - Sheet.csv")

In [4]:
train.head()

,img_name,label
0,25.jpg,25
1,27.jpg,27
2,28.jpg,28
3,30.jpg,30
4,30(1).jpg,30


In [5]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [6]:
train_data = train_datagen.flow_from_dataframe(train,directory="/kaggle/input/insat3d-infrared-raw-cyclone-images-20132021/insat3d_ir_cyclone_ds/CYCLONE_DATASET_INFRARED",subset="training",
                                               x_col="img_name",y_col="label",target_size=(512, 512),batch_size=16,class_mode='raw')

Found 136 validated image filenames.


In [7]:
def build_model():
    base = Xception(weights="/kaggle/input/cyclone-xception-weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5", include_top=False, input_tensor=Input(shape=(512, 512, 3)))

    base.trainable = False

    flatten = base.output
    flatten = Flatten()(flatten)

    bboxHead = Dense(64, activation="relu")(flatten)
    bboxHead = Dense(32, activation="relu")(bboxHead)
    bboxHead = Dense(1, activation="linear")(bboxHead)

    model = Model(inputs=base.input, outputs=bboxHead)
    
    return model

In [8]:
model = build_model()

In [9]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mae', metrics=[tf.keras.metrics.RootMeanSquaredError()])

save_best = tf.keras.callbacks.ModelCheckpoint("Model.h5", monitor='loss',save_best_only=True, verbose=1)

In [10]:
model.fit(train_data, epochs=50, callbacks=[save_best])

Epoch 1/50
9/9 [==============================] - ETA: 0s - loss: 27.4030 - root_mean_squared_error: 34.9818
Epoch 1: loss improved from inf to 27.40301, saving model to Model.h5
9/9 [==============================] - 24s 692ms/step - loss: 27.4030 - root_mean_squared_error: 34.9818
Epoch 2/50
9/9 [==============================] - ETA: 0s - loss: 14.8546 - root_mean_squared_error: 19.3992
Epoch 2: loss improved from 27.40301 to 14.85463, saving model to Model.h5
9/9 [==============================] - 4s 500ms/step - loss: 14.8546 - root_mean_squared_error: 19.3992
Epoch 3/50
9/9 [==============================] - ETA: 0s - loss: 10.8709 - root_mean_squared_error: 14.3573
Epoch 3: loss improved from 14.85463 to 10.87093, saving model to Model.h5
9/9 [==============================] - 5s 567ms/step - loss: 10.8709 - root_mean_squared_error: 14.3573
Epoch 4/50
9/9 [==============================] - ETA: 0s - loss: 8.0228 - root_mean_squared_error: 10.7522
Epoch 4: loss improved from 10.8

In [11]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [12]:
download_file('/kaggle/working/Model.h5', 'Kaggle_model')

/kaggle/working/Kaggle_model.zip